In [13]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [1]:
from past.utils import old_div

import os
import sys
from time import sleep
import h5py
from scipy import optimize
from scipy.optimize import curve_fit
import cv2
from time import sleep
import time
from IPython.display import display, clear_output
method = eval('cv2.TM_CCOEFF_NORMED')


In [3]:
cd ../../code

c:\Spectradyne\code


In [4]:
def rotate_image(image, angle):
    image_center = tuple(old_div(np.array(image.shape[1::-1]), 2))
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [5]:
import spectradyne.util.util_image_processing as image_processing

In [6]:
def get_nc_coords(input_image, input_template,titletext):
    """use open CV correlation method to match known template to image,
    assume NC is center of template, but now include +- 5degrees of
    rotation if needed.
    Note we're now (10/27/20) normalizing the image by removing the offest and fitting into  8 bits
    """
    method = eval('cv2.TM_CCOEFF_NORMED')
    imgn = input_image.copy()
    imgn = imgn - np.min(imgn)
    imgn = imgn * (old_div(256., np.max(imgn)))
    img = np.uint8(imgn)
    template = np.uint8(input_template.copy())
    w, h = template.shape[::-1]
    alist = []
    cclist = []
    for a in range(-5, 5, 1):
        rtemplate = rotate_image(template, a)
        res = cv2.matchTemplate(img, rtemplate, method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(np.abs(res))
        alist.append(a)
        cclist.append(max_val)
        #print('angle, degrees, NC coords,cc',a,max_loc,max_val)
    best_angle = alist[cclist.index(max(cclist))]
    rtemplate = rotate_image(template, best_angle)
    # Apply template Matching
    res = cv2.matchTemplate(img, rtemplate, method)
    figure()
    title(titletext)
    imshow(res)
    savefig(titletext+'.png')
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(np.abs(res))
    # print(min_val,max_val)
    nc_xy_ccval = (max_loc[0] + old_div(w, 2), max_loc[1] + old_div(h, 2), max_val)
    print(nc_xy_ccval)
    return res,nc_xy_ccval  # This is a 3-component tuple

In [7]:
def make_template(image, nc_xy_tuple):
   
    template = image[nc_xy_tuple[1] - 50:nc_xy_tuple[1] + 50, nc_xy_tuple[0] - 240:nc_xy_tuple[0] + 240, 1]
    cut=0.7*(np.max(template)-np.min(template))
    bintemplate=template-np.min(template)
    bintemplate[bintemplate <cut]=0
    bintemplate[bintemplate >= cut]=1
    bintemplate[:,195:285]=0
    
    return template

In [218]:
figure()
imshow(tmp)

In [242]:
tmp.shape

(100, 480)

In [238]:
figure()
diffs_low=abs((1.0*tmp[:50,75] - 1.0*tmp[4:54,75]))
diffs_high=abs((1.0*tmp[50:-4,75] - 1.0*tmp[54:,75]))

In [14]:
ncxy=(704,484)
tmp400=make_template(c400,ncxy)

In [15]:
print(np.argmax(diffs_low))
print(np.argmax(diffs_high)+50)


NameError: name 'diffs_low' is not defined

In [16]:
%pylab
matplotlib.rcParams.update({'font.size': 22})

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [13]:
f.close()

In [17]:
figure()
imshow(c400[:,:,1])

In [18]:
fc400= h5py.File('C://Spectradyne//data//template_tests//NC_c400_1_20220511.h5','r')
fc2000= h5py.File('C://Spectradyne//data//template_tests//NC_c2000_1_20220511.h5','r')
ft400=h5py.File('C://Spectradyne//reffiles//nc_templates/NC_template_c400.h5','r')
ft2000=h5py.File('C://Spectradyne//reffiles//nc_templates/NC_template_c2000.h5','r')

In [93]:
fc2000.keys()

<KeysViewHDF5 ['Laser_data']>

In [19]:
t2000=ft2000['template'].value
t400=ft400['template'].value
c400=fc400['Laser_data'].value
c2000=fc2000['Laser_data'].value


In [20]:
get_nc_coords(c400[:,:,1],t400)
get_nc_coords(c2000[:,:,1],t2000)
get_nc_coords(c400[:,:,1],t2000)
get_nc_coords(c2000[:,:,1],t400)

TypeError: get_nc_coords() missing 1 required positional argument: 'titletext'

In [23]:
print(get_nc_coords(c400[:,:,1],t400blank,'C400 with C400 Template'))
print(get_nc_coords(c2000[:,:,1],t2000blank,'C2000 with C2000 Template'))
print(get_nc_coords(c400[:,:,1],t2000blank,'C400 with C2000 Template'))
print(get_nc_coords(c2000[:,:,1],t400blank,'C2000 with C400 Template'))

(704, 483, 0.5941752195358276)
(array([[ 0.00429458,  0.00325966,  0.0025232 , ...,  0.01129717,
         0.01164513,  0.01310185],
       [ 0.00481146,  0.00336327,  0.00238963, ...,  0.01220155,
         0.01355329,  0.01435778],
       [ 0.00660232,  0.00480769,  0.00329151, ...,  0.01450304,
         0.01528149,  0.0161702 ],
       ...,
       [-0.06435493, -0.05981618, -0.05440811, ...,  0.11752939,
         0.11743502,  0.11688828],
       [-0.06405526, -0.05899476, -0.05481724, ...,  0.11870895,
         0.11855455,  0.11832998],
       [-0.06379038, -0.0594117 , -0.05626228, ...,  0.11755481,
         0.11756562,  0.11737734]], dtype=float32), (704, 483, 0.5941752195358276))
(704, 492, 0.5977470278739929)
(array([[ 2.9335637e-03,  1.5731647e-03, -8.4824947e-05, ...,
         2.2186248e-02,  2.1402674e-02,  2.0908756e-02],
       [ 2.9783095e-03,  1.1193929e-03, -4.9769919e-04, ...,
         1.9858059e-02,  1.9500325e-02,  1.9184869e-02],
       [ 2.6339064e-03,  8.0108788e-04,

In [210]:
temp=remove_xchunk(t400blank,30)
#print(get_nc_coords(c2000[:,:,1],t400blank))
print(get_nc_coords(c2000[:,:,1],temp,'C2000 with C400 tmp,60 cols rmved'))

(701, 493, 0.5707237720489502)


In [22]:
t400blank=t400.copy()
t400blank[:,195:285]=0
t2000blank=t2000.copy()
t2000blank[:,195:285]=0

In [179]:
tst=remove_xchunk(t400blank,10)

In [205]:
cd c://Spectradyne//Data/template_tests

c:\Spectradyne\Data\template_tests


In [208]:
figure(figsize=[12,6])
title('C400 template with blanking')
imshow(t400blank)
savefig('t400blank.png')
t400blank_crop60=remove_xchunk(t400blank,30)

figure(figsize=[12,6])
title('C400 template with blanking, 60col cropped')
imshow(t400blank_crop60)
savefig('t400blank_crop60.png')


figure(figsize=[12,6])
title('C2000 template with blanking')
imshow(t2000blank)
savefig('t2000blank.png')

In [182]:
print(t400blank.shape)
figure()
imshow(tst)

(100, 480)


In [24]:
def remove_xchunk(intemplate,npix):
    #tool to cut out 2x npix from center (in x) of a template
    nx,ny=intemplate.shape
    ystart=int(ny/2) - npix
    ystop=int(ny/2) + npix
    outtemplate=np.concatenate([intemplate[:,:ystart],intemplate[:,ystop:]],axis=1)
    return outtemplate
    


In [25]:
cclist=[]
for i in range(15):
    temp=remove_xchunk(t400blank,3*i)
    x,y,cc=get_nc_coords(c2000[:,:,1],temp)
    cclist.append(cc)

TypeError: get_nc_coords() missing 1 required positional argument: 'titletext'

In [26]:
figure()
imshow(c2000[:,:,1])

In [199]:
figure()
title('Best cross correlation C2000 image, cropped C400 template')
xlabel('# of pixel columns cropped from template center')
ylabel('Max cross correlation value')
x=6*arange(15)
plot(x,cclist)


In [195]:
close('all')

In [133]:
figure()
imshow(c400[:,:,1])
figure()
imshow(c2000[:,:,1])

In [138]:
figure()
imshow(t2000blank)

In [81]:
t2000_temp=np.uint8(t2000_1.copy())
cc=cv2.matchTemplate(np.uint8(c2000_1),t2000_temp,method)

figure()
imshow(cc)

In [127]:
figure()
imshow(t400_temp)

In [27]:
def make_line_templatei(intemplate):
    '''attempt model based template construction, try for 4 whisker lines
    remember that x is second axis while y is first axis'''
    ysize,xsize=intemplate.shape
    ys2=np.int(ysize/2)
    lw=3 #line width in y direction
    xlows=[75,150,325,400]
    xhighs=xlows.copy()
    ylows=[]
    yhighs=[]
    for x in xlows:
        ylows.append(np.argmax(abs(1.0*intemplate[:ys2,x] - 1.0*intemplate[4:ys2+4,x])))
        yhighs.append(ys2+4+np.argmax(abs(1.0*intemplate[ys2:-4,x] - 1.0*intemplate[ys2+4:,x])))
        
    outtemplate=intemplate.copy()
    outtemplate[:,:]=0
    xlow1=(arange(xlows[0],xlows[1])).astype(int)
    ylow1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[ylows[0],ylows[1]])).astype(int)
    ylow1a=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[ylows[0]-1,ylows[1]-1])).astype(int)
    ylow1b=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[ylows[0]+1,ylows[1]+1])).astype(int)
    
    xlow2=(arange(xlows[2],xlows[3])).astype(int)
    ylow2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[ylows[2],ylows[3]])).astype(int)
    ylow2a=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[ylows[2]-1,ylows[3]-1])).astype(int)
    ylow2b=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[ylows[2]+1,ylows[3]+1])).astype(int)
    
    xhigh1=(arange(xlows[0],xlows[1])).astype(int)
    yhigh1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[yhighs[0],yhighs[1]])).astype(int)
    yhigh1a=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[yhighs[0]-1,yhighs[1]-1])).astype(int)
    yhigh1b=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[yhighs[0]+1,yhighs[1]+1])).astype(int)
    
    xhigh2=(arange(xlows[2],xlows[3])).astype(int)
    yhigh2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[yhighs[2],yhighs[3]])).astype(int)
    yhigh2a=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[yhighs[2]-1,yhighs[3]-1])).astype(int)
    yhigh2b=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[yhighs[2]+1,yhighs[3]+1])).astype(int)   
    
    for x,y in zip(xlow1,ylow1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y,x]=1
    for x,y in zip(xlow1,yhigh1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,yhigh2):
        outtemplate[y,x]=1
        
    for x,y in zip(xlow1,ylow1a):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,ylow2a):
        outtemplate[y,x]=1
    for x,y in zip(xlow1,yhigh1a):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,yhigh2a):
        outtemplate[y,x]=1

    for x,y in zip(xlow1,ylow1b):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,ylow2b):
        outtemplate[y,x]=1
    for x,y in zip(xlow1,yhigh1b):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,yhigh2b):
        outtemplate[y,x]=1
        
    return outtemplate
    

In [28]:
def make_line_template_original(intemplate):
    '''attempt model based template construction, try for 4 whisker lines
    remember that x is second axis while y is first axis'''
    ysize,xsize=intemplate.shape
    jog=2
    ys2=np.int(ysize/2)
    lw=3 #line width in y direction
    xlows=[75,170,325,425]
    xhighs=xlows.copy()
    ylows=[]
    yhighs=[]
    for x in xlows:
        ylows.append(np.argmax(abs(1.0*intemplate[:ys2,x] - 1.0*intemplate[4:ys2+4,x])))
        yhighs.append(ys2+4+np.argmax(abs(1.0*intemplate[ys2:-4,x] - 1.0*intemplate[ys2+4:,x])))
        
    outtemplate=intemplate.copy()
    outtemplate[:,:]=0
    xlow1=(arange(xlows[0],xlows[1])).astype(int)
    ylow1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[ylows[0],ylows[1]])).astype(int)
    
    xlow2=(arange(xlows[2],xlows[3])).astype(int)
    ylow2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[ylows[2],ylows[3]])).astype(int)
    
    xhigh1=(arange(xlows[0],xlows[1])).astype(int)
    yhigh1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[yhighs[0],yhighs[1]])).astype(int)
    
    xhigh2=(arange(xlows[2],xlows[3])).astype(int)
    yhigh2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[yhighs[2],yhighs[3]])).astype(int)
    
    
    for x,y in zip(xlow1,ylow1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y,x]=1
    for x,y in zip(xlow1,yhigh1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,yhigh2):
        outtemplate[y,x]=1
        
    for x,y in zip(xlow1,ylow1):
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
    for x,y in zip(xlow1,yhigh1):
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
    for x,y in zip(xlow2,yhigh2):
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
            
    for x,y in zip(xlow1,ylow1):
        outtemplate[y-2,x]=1
        #outtemplate[y+2,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y-2,x]=1
        #outtemplate[y+2,x]=1
    for x,y in zip(xlow1,yhigh1):
        #outtemplate[y-2,x]=1
        outtemplate[y+2,x]=1
    for x,y in zip(xlow2,yhigh2):
        #outtemplate[y-2,x]=1
        outtemplate[y+2,x]=1
        
    print(get_cc(intemplate,outtemplate))
        
    return outtemplate
    

In [29]:
def make_line_template(intemplate):
    '''model based template construction, try for 4 whisker lines
    remember that x is second axis while y is first axis'''
    ysize,xsize=intemplate.shape
    jog=1
    ys2=np.int(ysize/2)
    lw=3 #line width in y direction
    xlows=[75,170,325,425]
    xhighs=xlows.copy()
    ylows=[]
    yhighs=[]
    for x in xlows:
        ylows.append(np.argmax(abs(1.0*intemplate[:ys2,x] - 1.0*intemplate[4:ys2+4,x])))
        yhighs.append(ys2+8+np.argmax(abs(1.0*intemplate[ys2:-4,x] - 1.0*intemplate[ys2+4:,x])))
       
    outtemplate=fill_template(intemplate,xlows,ylows,yhighs)
    print('yhighs',yhighs)
    print('ylows',ylows)
    print(get_cc(intemplate,outtemplate))
    return outtemplate
    

In [30]:
def fill_template(intemplate,xlows,ylows,yhighs, offsets=[0,0,0,0,0,0,0,0]):
    lo=offsets[:4]
    ho=offsets[4:]
    outtemplate=intemplate.copy()
    outtemplate[:,:]=0
    xlow1=(arange(xlows[0],xlows[1])).astype(int)
    ylow1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[ylows[0]+lo[0],ylows[1]+lo[1]])).astype(int)
    
    xlow2=(arange(xlows[2],xlows[3])).astype(int)
    ylow2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[ylows[2]+lo[2],ylows[3]+lo[3]])).astype(int)
    
    xhigh1=(arange(xlows[0],xlows[1])).astype(int)
    yhigh1=(np.interp(arange(xlows[0],xlows[1]),[xlows[0],xlows[1]],[yhighs[0]+ho[0],yhighs[1]+ho[1]])).astype(int)
    
    xhigh2=(arange(xlows[2],xlows[3])).astype(int)
    yhigh2=(np.interp(arange(xlows[2],xlows[3]),[xlows[2],xlows[3]],[yhighs[2]+ho[2],yhighs[3]+ho[3]])).astype(int)
    
    for x,y in zip(xlow1,ylow1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y,x]=1
    for x,y in zip(xlow1,yhigh1):
        outtemplate[y,x]=1
    for x,y in zip(xlow2,yhigh2):
        outtemplate[y,x]=1
#thicken by adding another 2 lines of pixels above and below        
    for x,y in zip(xlow1,ylow1):
        outtemplate[y-2,x]=1
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
        outtemplate[y+2,x]=1
        outtemplate[y-3,x]=1
        outtemplate[y+3,x]=1
    for x,y in zip(xlow2,ylow2):
        outtemplate[y-2,x]=1
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
        outtemplate[y+2,x]=1
        outtemplate[y-3,x]=1
        outtemplate[y+3,x]=1
    for x,y in zip(xlow1,yhigh1):
        outtemplate[y-2,x]=1
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
        outtemplate[y+2,x]=1
        outtemplate[y-3,x]=1
        outtemplate[y+3,x]=1
    for x,y in zip(xlow2,yhigh2):
        outtemplate[y-2,x]=1
        outtemplate[y-1,x]=1
        outtemplate[y+1,x]=1
        outtemplate[y+2,x]=1
        outtemplate[y-3,x]=1
        outtemplate[y+3,x]=1
            
    return outtemplate

In [31]:
def make_line_template_tune(intemplate):
    '''attempt model based template construction, try for 4 whisker lines
    remember that x is second axis while y is first axis'''
    ysize,xsize=intemplate.shape
    jog=1
    ys2=np.int(ysize/2)
    lw=3 #line width in y direction
    xlows=[75,170,325,425]
    xhighs=xlows.copy()
    ylows=[]
    yhighs=[]
    for x in xlows:
        ylows.append(np.argmax(abs(1.0*intemplate[:ys2,x] - 1.0*intemplate[4:ys2+4,x])))
        yhighs.append(ys2+8+np.argmax(abs(1.0*intemplate[ys2:-4,x] - 1.0*intemplate[ys2+4:,x])))
       
    
    ilist=[]; jlist=[]; klist=[]; llist=[]; mlist=[]; nlist=[]; olist=[]; plist=[];
    i=0; j=0; k=0; l=0; m=0; n=0; o=0; p=0
    #line 1
    cclist=[]
    for i in arange(5)-3:
        for j in arange(5)-3:
            ilist.append(i); jlist.append(j)
            offsets=[n for n in [i,j,k,l,m,n,o,p]]
            outtemplate=fill_template(intemplate,xlows,ylows,yhighs,offsets)
            xx,yy,cc=get_cc(intemplate,outtemplate)
            cclist.append(cc)
    indmax=np.argmax(cclist)
    ibest=ilist[indmax]
    jbest=jlist[indmax]

    #line 2
    cclist=[]
    for k in arange(5)-3:
        for l in arange(5)-3:
            klist.append(k); llist.append(l)
            offsets=[n for n in [ibest,jbest,k,l,m,n,o,p]]
            outtemplate=fill_template(intemplate,xlows,ylows,yhighs,offsets)
            xx,yy,cc=get_cc(intemplate,outtemplate)
            cclist.append(cc)
    indmax=np.argmax(cclist)
    kbest=klist[indmax]
    lbest=llist[indmax]
    
    #line 3
    cclist=[]
    for m in arange(5)-3:
        for n in arange(5)-3:
            mlist.append(m); nlist.append(n)
            offsets=[n for n in [ibest,jbest,kbest,lbest,m,n,o,p]]
            outtemplate=fill_template(intemplate,xlows,ylows,yhighs,offsets)
            xx,yy,cc=get_cc(intemplate,outtemplate)
            cclist.append(cc)
    indmax=np.argmax(cclist)
    mbest=mlist[indmax]
    nbest=nlist[indmax]

    #line 4
    cclist=[]
    for o in arange(5)-3:
        for p in arange(5)-3:
            olist.append(o); plist.append(p)
            offsets=[n for n in [ibest,jbest,kbest,lbest,mbest,nbest,o,p]]
            outtemplate=fill_template(intemplate,xlows,ylows,yhighs,offsets)
            xx,yy,cc=get_cc(intemplate,outtemplate)
            cclist.append(cc)
    indmax=np.argmax(cclist)
    obest=olist[indmax]
    pbest=plist[indmax]
    offsets=[n for n in [ibest,jbest,kbest,lbest,mbest,nbest,obest,pbest]]
    outtemplate=fill_template(intemplate,xlows,ylows,yhighs,offsets)
    print('offsets = ',offsets)
    print(get_cc(intemplate,outtemplate))
    
    return outtemplate
    

In [34]:
outtemp6=make_line_template_tune(tmp400)

offsets =  [-1, -1, -1, -1, -2, -2, -3, -2]
(240, 50, 0.5622978210449219)


In [473]:
print(np.max(cclist))
imax=np.argmax(cclist)
print(np.min(cclist))
imin=np.argmin(cclist)

0.5243163108825684
0.000791213707998395


In [475]:
offsetlist[imax]


[0, 0, 0, 0, 0, -4, -4, 0]

In [457]:
shape(cclist)

(6561,)

In [35]:
linetmp400b=make_line_template(tmp400)

yhighs [76, 63, 70, 89]
ylows [12, 35, 35, 16]
(240, 50, 0.4750243127346039)


In [40]:
linetmp400=make_line_template(tmp400)

yhighs [76, 63, 70, 89]
ylows [12, 35, 35, 16]
(240, 50, 0.4750243127346039)


In [39]:
linetmp400i=make_line_templatei(tmp400)

In [361]:
ax[0]

11

In [37]:
ay,ax=where(linetmp400==1)

NameError: name 'linetmp400' is not defined

In [367]:
figure()
imshow(linetmp400)

In [296]:
close('all')

In [42]:
get_nc_coords(c400[:,:,1],linetmp400i,'linetemp400i')

(705, 485, 0.3922620713710785)


(array([[ 0.01705409,  0.01550127,  0.0129027 , ..., -0.00174351,
          0.00035246,  0.00355184],
        [ 0.01676688,  0.01580356,  0.01361328, ..., -0.00196746,
          0.00062709,  0.00414922],
        [ 0.01673139,  0.0157635 ,  0.01387372, ..., -0.00200903,
          0.00090724,  0.00460692],
        ...,
        [-0.01867857, -0.0151755 , -0.01177265, ...,  0.01425054,
          0.01424238,  0.01435431],
        [-0.01923337, -0.01706425, -0.01398818, ...,  0.01228735,
          0.01265365,  0.01320272],
        [-0.01931428, -0.01736264, -0.01439548, ...,  0.0100082 ,
          0.01037881,  0.01108919]], dtype=float32),
 (705, 485, 0.3922620713710785))

In [43]:
get_nc_coords(c400[:,:,1],linetmp400,'linetemp400')

(704, 482, 0.5348321199417114)


(array([[-0.01052925, -0.00899566, -0.0074416 , ...,  0.00774584,
          0.01103778,  0.01548913],
        [-0.01338569, -0.01228713, -0.01094524, ...,  0.0071494 ,
          0.01076345,  0.01560222],
        [-0.01648308, -0.01588407, -0.01491169, ...,  0.00666118,
          0.0108855 ,  0.01625243],
        ...,
        [ 0.00960986,  0.0099442 ,  0.01229947, ...,  0.02058761,
          0.02110181,  0.02149974],
        [ 0.00851077,  0.00958203,  0.01353565, ...,  0.02024523,
          0.0209155 ,  0.02186426],
        [ 0.00536805,  0.00679908,  0.01034573, ...,  0.02040643,
          0.02119958,  0.02212022]], dtype=float32),
 (704, 482, 0.5348321199417114))

In [46]:
imshow(linetmp400)

In [45]:
figure()
imshow(t400)

In [317]:
c400[:,:,1].shape

(1080, 1440)

In [33]:
def get_cc(img,template):
    method = eval('cv2.TM_CCOEFF_NORMED')
    imgn = img.copy()
    imgn = imgn - np.min(imgn)
    imgn = imgn * (old_div(256., np.max(imgn)))
    img = np.uint8(imgn)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img, np.uint8(template), method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(np.abs(res))
    nc_xy_ccval = (max_loc[0] + old_div(w, 2), max_loc[1] + old_div(h, 2), max_val)
    return nc_xy_ccval

In [337]:
print(get_cc(tmp400,linetmp400))

(240, 50, 0.4411216676235199)


In [405]:
figure()
imshow(t400blank)

In [334]:
tmp400.shape

(100, 480)

In [52]:
res_l400_c400,xycc=get_nc_coords(c400[:,:,1],outtemp_minus,'C400 with c400 inverted tuned line template')

(704, 483, 0.8123605847358704)


In [52]:
res_t400,xycc=get_nc_coords(c400[:,:,1],t400,'C400 with c400 nominal template')

(704, 483, 0.8123605847358704)


In [55]:
figure()
plot(res_l400_c400.flatten()/np.max(res_l400_c400),label='C400 CC from line template')
plot(res_t400.flatten()/np.max(res_t400),label='C400 CCC from normal template')
xlabel('Pixel')
ylabel('Cross correlation value (normalized)')
legend()

In [409]:
figure()
imshow (res_l400_c2000)

In [411]:
figure()
imshow(t2000+linetmp400)

In [422]:
figure()
imshow(t400+linetmp400b)

In [48]:
figure()
title('c400 template area with tuned line template overlay')
imshow(tmp400+outtemp6*1000)

In [49]:
outtemp_minus=(outtemp6-1.0).astype(int)

In [50]:
figure()
imshow(outtemp_minus)